In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import shutil
import glob

# 경고 뜨지 않게...
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False
# 매직명령어 => 쥬피터노트북에서 그래프 삽입 기능 
%matplotlib inline
# 글꼴 선명화 
%config InlineBackend.figure_format = 'retina'

# 랜덤 모듈
import random

# 학습 모델 저장 및 복원
import pickle

# 딥러닝 라이브러리
import tensorflow as tf
# 신경망 모델을 관리하는 객체
from tensorflow.keras.models import Sequential
# 선형 회귀 레이어
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 객체
from tensorflow.keras.layers import Activation

# CNN : 커널을 통해 합성곱을 구하는 것. 이미지의 특징이 두드러 지게 한다.
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv1D

# MaxPooling : 커널내에서 가장 큰 값을 추출하는 방식으로 불필요한 부분을 제거한다.
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import MaxPool1D

# 1차원으로 변환하는 것
from tensorflow.keras.layers import Flatten

# Dropout : 이미자나 영상, 음파 등의 데이터는 오랫동안 학습을 진행 시켜야 한다.
# 하지만 너무 빨리 과적합이 발생되면 조기 중단 때문에 학습 횟수가 줄어들 수 있다.
# 이에 은닉의 노드를 일부 사용하지 않으므로써 과적합이 빨리 오는 것을 예방하고
# 오랫동안 학습이 가능하다.
from tensorflow.keras.layers import Dropout

# Embadding : 단어의 수를 조정한다.
from tensorflow.keras.layers import Embedding

# RNN(LSTM)
from tensorflow.keras.layers import LSTM

# GAN
# 데이터 표준화
from tensorflow.keras.layers import BatchNormalization
# 데이터를 다시 구성하기 위한 것
from tensorflow.keras.layers import Reshape
# 업샘플링
from tensorflow.keras.layers import UpSampling2D
# 이미지 생성을 위해 사용할 데이터 공간을 생성
from tensorflow.keras.layers import Input
# 다수의 신경망을 연결하기 위한 것(생성자와 판별자를 연결)
from tensorflow.keras.models import Model

# 원핫 인코딩을 수행하는 함수
from tensorflow.keras.utils import to_categorical

# 저장된 학습모델을 복원한다.
from tensorflow.keras.models import load_model

# 모델을 자동 저장한다.
from tensorflow.keras.callbacks import ModelCheckpoint
# 성능이 더이상 좋아지지 않을 경우 중단 시킨다.
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

# 문장을 잘라준다.
from tensorflow.keras.preprocessing.text import Tokenizer
# 모든 문장 데이터의 단어 데이터의 수를 동일한 수로 맞춰준다.
from tensorflow.keras.preprocessing.sequence import pad_sequences
# 문자열을 가지고 단어 사전을 만들어준다.
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 표준화
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# 문자열 => 숫자
from sklearn.preprocessing import LabelEncoder

# 전체데이터를 학습용과 검증으로 나눈다.
from sklearn.model_selection import train_test_split

# 이미지 파일을 지정하여 이미지 데이터를 읽어온다.
from PIL import Image

# 랜덤시드 설정
# 데이터를 랜덤하게 섞거나 가중치를 랜덤하게 설정하는 등..
# 작업에서 랜덤을 적용하는 경우가 더러 있다.
# 이에, 시드를 고정시킨다.
random_seed = 1
np.random.seed(random_seed)
random.seed(random_seed)
tf.random.set_seed(random_seed)

# 현재 프로젝트에서 GPU 메모리 사용을 필요한 만큼만 쓸 수 있도록 한다.
# 컴퓨터에 있는 GPU 정보들을 가져온다.

gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면...
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

In [2]:
# 하위 폴더들의 이름을 가져온다(결과 데이터)
categories = list(os.walk('./image/caltech-101/101_ObjectCategories'))[0][1]
categories

['accordion',
 'airplanes',
 'anchor',
 'ant',
 'barrel',
 'bass',
 'beaver',
 'binocular',
 'bonsai',
 'brain',
 'brontosaurus',
 'buddha',
 'butterfly',
 'camera',
 'cannon',
 'car_side',
 'ceiling_fan',
 'cellphone',
 'chair',
 'chandelier',
 'cougar_body',
 'cougar_face',
 'crab',
 'crayfish',
 'crocodile',
 'crocodile_head',
 'cup',
 'dalmatian',
 'dollar_bill',
 'dolphin',
 'dragonfly',
 'electric_guitar',
 'elephant',
 'emu',
 'euphonium',
 'ewer',
 'Faces',
 'Faces_easy',
 'ferry',
 'flamingo',
 'flamingo_head',
 'garfield',
 'gerenuk',
 'gramophone',
 'grand_piano',
 'hawksbill',
 'headphone',
 'hedgehog',
 'helicopter',
 'ibis',
 'inline_skate',
 'joshua_tree',
 'kangaroo',
 'ketch',
 'lamp',
 'laptop',
 'Leopards',
 'llama',
 'lobster',
 'lotus',
 'mandolin',
 'mayfly',
 'menorah',
 'metronome',
 'minaret',
 'Motorbikes',
 'nautilus',
 'octopus',
 'okapi',
 'pagoda',
 'panda',
 'pigeon',
 'pizza',
 'platypus',
 'pyramid',
 'revolver',
 'rhino',
 'rooster',
 'saxophone',
 'sc

In [3]:
# 이미지 파일들을 가지고 있는 최상위 경로
caltech_dir = './image/caltech-101/101_ObjectCategories'
# 전체 카테고리 개수
nb_class = len(categories)

In [4]:
# 변형될 이미지의 크기
image_w = 64
image_h = 64
image_channel = 3
# 전체 픽셀의 개수
pixels = image_h * image_w * image_channel

In [5]:
pixels

12288

In [6]:
# 이미지 변형작업
# 이미지들의 데이터를 담을 리스트
X = []
# 결과데이터(원핫인코딩된)를 담을 리스트
y = []

# 카테고리 개수 만큼 반복한다.
for idx, cat in enumerate(categories) :
    # print(idx, cat)
    
    # 결과데이터 원핫 인코딩
    label = []
    for _ in range(nb_class) :
        label.append(0)
    label[idx] = 1
    
    # print(label)
    
    # 이미지를 읽어올 경로
    image_dir = caltech_dir + '/' + cat
    # print(image_dir)
    
    # 경로 내의 이미지 파일의 이름을 가져온다.
    files = glob.glob(image_dir + '/＊.jpg')
    # print(files)
    
    # 가져온 이미지 파일명의 수 만큼 반복한다.
    for i, f in enumerate(files):
        # 이미지를 읽어온다.
        img = Image.open(f)
        # CNN을 사용할 것이기 때문에 데이터를 RGB형태로 변환한다.
        # jpg 이미지는 데이터가 GBR 형태로 되어있다.
        img = img.convert('RGB')
        # 이미지의 크기를 조정한다.
        img = img.resize((image_w, image_h))
        # 행렬로 변환한다.
        data = np.array(img)
        # 리스트에 담는다.
        X.append(data)
        y.append(label)
        
    print(f'{cat} 작업완료')

accordion 작업완료
airplanes 작업완료
anchor 작업완료
ant 작업완료
barrel 작업완료
bass 작업완료
beaver 작업완료
binocular 작업완료
bonsai 작업완료
brain 작업완료
brontosaurus 작업완료
buddha 작업완료
butterfly 작업완료
camera 작업완료
cannon 작업완료
car_side 작업완료
ceiling_fan 작업완료
cellphone 작업완료
chair 작업완료
chandelier 작업완료
cougar_body 작업완료
cougar_face 작업완료
crab 작업완료
crayfish 작업완료
crocodile 작업완료
crocodile_head 작업완료
cup 작업완료
dalmatian 작업완료
dollar_bill 작업완료
dolphin 작업완료
dragonfly 작업완료
electric_guitar 작업완료
elephant 작업완료
emu 작업완료
euphonium 작업완료
ewer 작업완료
Faces 작업완료
Faces_easy 작업완료
ferry 작업완료
flamingo 작업완료
flamingo_head 작업완료
garfield 작업완료
gerenuk 작업완료
gramophone 작업완료
grand_piano 작업완료
hawksbill 작업완료
headphone 작업완료
hedgehog 작업완료
helicopter 작업완료
ibis 작업완료
inline_skate 작업완료
joshua_tree 작업완료
kangaroo 작업완료
ketch 작업완료
lamp 작업완료
laptop 작업완료
Leopards 작업완료
llama 작업완료
lobster 작업완료
lotus 작업완료
mandolin 작업완료
mayfly 작업완료
menorah 작업완료
metronome 작업완료
minaret 작업완료
Motorbikes 작업완료
nautilus 작업완료
octopus 작업완료
okapi 작업완료
pagoda 작업완료
panda 작업완료
pigeon 작업완료
pizza 작업완료
platy

In [11]:
# 이미지 변형작업

# 이미지들의 데이터를 담을 리스트
X = []
# 결과데이터(원핫인코딩된)를 담을 리스트
y = []

# 카테고리 개수 만큼 반복한다.
for idx, cat in enumerate(categories) :
    # print(idx, cat)
    
    # 결과데이터 원핫 인코딩
    label = []
    for _ in range(nb_class) :
        label.append(0)
    label[idx] = 1
    
    # print(label)
    
    # 이미지를 읽어올 경로
    image_dir = caltech_dir + '/' + cat
    # print(image_dir)
    
    # 경로내의 이미지 파일 이름을 가져온다.
    files = glob.glob(image_dir + '/*.jpg')
    # print(files)
    
    # 가져온 이미지 파일명의 수 만큼 반복한다.
    for i, f in enumerate(files) :
        # 이미지를 읽어온다.
        img = Image.open(f)
        # CNN을 사용할 것이기 때문에 데이터를 RGB형태로 변환한다.
        # jpg 이미지는 데이터가 GBR 형태로 되어 있다.
        img = img.convert('RGB')
        # 이미지의 크기를 조정한다.
        img = img.resize((image_w, image_h))
        # 행렬로 변환한다.
        data = np.array(img)
        # 리스트에 담는다.
        X.append(data)
        y.append(label)
    
    print(f'{cat} 작업완료')


accordion 작업완료
airplanes 작업완료
anchor 작업완료
ant 작업완료
barrel 작업완료
bass 작업완료
beaver 작업완료
binocular 작업완료
bonsai 작업완료
brain 작업완료
brontosaurus 작업완료
buddha 작업완료
butterfly 작업완료
camera 작업완료
cannon 작업완료
car_side 작업완료
ceiling_fan 작업완료
cellphone 작업완료
chair 작업완료
chandelier 작업완료
cougar_body 작업완료
cougar_face 작업완료
crab 작업완료
crayfish 작업완료
crocodile 작업완료
crocodile_head 작업완료
cup 작업완료
dalmatian 작업완료
dollar_bill 작업완료
dolphin 작업완료
dragonfly 작업완료
electric_guitar 작업완료
elephant 작업완료
emu 작업완료
euphonium 작업완료
ewer 작업완료
Faces 작업완료
Faces_easy 작업완료
ferry 작업완료
flamingo 작업완료
flamingo_head 작업완료
garfield 작업완료
gerenuk 작업완료
gramophone 작업완료
grand_piano 작업완료
hawksbill 작업완료
headphone 작업완료
hedgehog 작업완료
helicopter 작업완료
ibis 작업완료
inline_skate 작업완료
joshua_tree 작업완료
kangaroo 작업완료
ketch 작업완료
lamp 작업완료
laptop 작업완료
Leopards 작업완료
llama 작업완료
lobster 작업완료
lotus 작업완료
mandolin 작업완료
mayfly 작업완료
menorah 작업완료
metronome 작업완료
minaret 작업완료
Motorbikes 작업완료
nautilus 작업완료
octopus 작업완료
okapi 작업완료
pagoda 작업완료
panda 작업완료
pigeon 작업완료
pizza 작업완료
platy

In [13]:
# 리스트를 ndarray로 생성한다.
X = np.array(X)
y = np.array(y)

In [14]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [15]:
# 학습용과 검증용으로 나눈다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [16]:
# 저장을 위해 하나의 튜플로 생성한다.
xy = (X_train, X_test, y_train, y_test)
# 저장
np.save('./image/data1.npy', xy)